In [29]:
from __future__ import print_function, division

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam,RMSprop

import tensorflow.keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np


class WGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        self.clip_value = 0.005
        optimizer = RMSprop(lr=0.00005)

        # Build and compile the critic
        self.critic = self.build_critic()
        self.critic.compile(loss=self.wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.critic.trainable = False

        # The critic takes generated images as input and determines validity
        valid = self.critic(img)

        # The combined model  (stacked generator and critic)
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.wasserstein_loss,
            optimizer=optimizer)

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        x=2
        model.add(Conv2D(x*16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(x*32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(x*64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(x*128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))

        for epoch in range(epochs):

            self.critic.trainable = True
            for _ in range(self.n_critic):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random batch of images
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                imgs = X_train[idx]
                
                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the critic
                d_loss_real = self.critic.train_on_batch(imgs, valid)
                d_loss_fake = self.critic.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

                # Clip critic weights
                for l in self.critic.layers:
                    weights = l.get_weights()
                    weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                    l.set_weights(weights)


            # ---------------------
            #  Train Generator
            # ---------------------

            #self.critic.trainable = False
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            #print(epoch)
            print ("%d [D loss: %f] [G loss: %f]" % (epoch, 1 - d_loss[0], 1 - g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/wgan_mnist_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    wgan = WGAN()
wgan.train(epochs=4000, batch_size=32, sample_interval=50)

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_116 (Conv2D)          (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_68 (LeakyReLU)   (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_68 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_117 (Conv2D)          (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_17 (ZeroPaddi (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_83 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_69 (LeakyReLU)   (None, 8, 8, 64)        

86 [D loss: 0.999970] [G loss: 1.000065]
87 [D loss: 0.999972] [G loss: 1.000068]
88 [D loss: 0.999969] [G loss: 1.000063]
89 [D loss: 0.999970] [G loss: 1.000067]
90 [D loss: 0.999970] [G loss: 1.000064]
91 [D loss: 0.999969] [G loss: 1.000066]
92 [D loss: 0.999970] [G loss: 1.000067]
93 [D loss: 0.999971] [G loss: 1.000067]
94 [D loss: 0.999969] [G loss: 1.000066]
95 [D loss: 0.999971] [G loss: 1.000066]
96 [D loss: 0.999970] [G loss: 1.000066]
97 [D loss: 0.999969] [G loss: 1.000065]
98 [D loss: 0.999971] [G loss: 1.000065]
99 [D loss: 0.999969] [G loss: 1.000063]
100 [D loss: 0.999970] [G loss: 1.000064]
101 [D loss: 0.999972] [G loss: 1.000067]
102 [D loss: 0.999972] [G loss: 1.000065]
103 [D loss: 0.999972] [G loss: 1.000064]
104 [D loss: 0.999971] [G loss: 1.000062]
105 [D loss: 0.999969] [G loss: 1.000063]
106 [D loss: 0.999972] [G loss: 1.000069]
107 [D loss: 0.999970] [G loss: 1.000062]
108 [D loss: 0.999971] [G loss: 1.000067]
109 [D loss: 0.999971] [G loss: 1.000065]
110 [D

282 [D loss: 0.999970] [G loss: 1.000065]
283 [D loss: 0.999970] [G loss: 1.000065]
284 [D loss: 0.999969] [G loss: 1.000065]
285 [D loss: 0.999967] [G loss: 1.000063]
286 [D loss: 0.999971] [G loss: 1.000066]
287 [D loss: 0.999971] [G loss: 1.000067]
288 [D loss: 0.999971] [G loss: 1.000067]
289 [D loss: 0.999971] [G loss: 1.000064]
290 [D loss: 0.999969] [G loss: 1.000063]
291 [D loss: 0.999970] [G loss: 1.000062]
292 [D loss: 0.999970] [G loss: 1.000065]
293 [D loss: 0.999972] [G loss: 1.000065]
294 [D loss: 0.999973] [G loss: 1.000066]
295 [D loss: 0.999972] [G loss: 1.000065]
296 [D loss: 0.999970] [G loss: 1.000065]
297 [D loss: 0.999971] [G loss: 1.000063]
298 [D loss: 0.999970] [G loss: 1.000065]
299 [D loss: 0.999967] [G loss: 1.000064]
300 [D loss: 0.999970] [G loss: 1.000064]
301 [D loss: 0.999972] [G loss: 1.000066]
302 [D loss: 0.999970] [G loss: 1.000064]
303 [D loss: 0.999971] [G loss: 1.000065]
304 [D loss: 0.999971] [G loss: 1.000063]
305 [D loss: 0.999971] [G loss: 1.

478 [D loss: 0.999969] [G loss: 1.000066]
479 [D loss: 0.999968] [G loss: 1.000067]
480 [D loss: 0.999970] [G loss: 1.000063]
481 [D loss: 0.999970] [G loss: 1.000067]
482 [D loss: 0.999970] [G loss: 1.000067]
483 [D loss: 0.999971] [G loss: 1.000067]
484 [D loss: 0.999971] [G loss: 1.000068]
485 [D loss: 0.999970] [G loss: 1.000064]
486 [D loss: 0.999971] [G loss: 1.000066]
487 [D loss: 0.999971] [G loss: 1.000065]
488 [D loss: 0.999972] [G loss: 1.000066]
489 [D loss: 0.999969] [G loss: 1.000065]
490 [D loss: 0.999968] [G loss: 1.000066]
491 [D loss: 0.999972] [G loss: 1.000065]
492 [D loss: 0.999971] [G loss: 1.000066]
493 [D loss: 0.999971] [G loss: 1.000066]
494 [D loss: 0.999969] [G loss: 1.000061]
495 [D loss: 0.999967] [G loss: 1.000066]
496 [D loss: 0.999973] [G loss: 1.000067]
497 [D loss: 0.999969] [G loss: 1.000069]
498 [D loss: 0.999971] [G loss: 1.000065]
499 [D loss: 0.999969] [G loss: 1.000063]
500 [D loss: 0.999971] [G loss: 1.000068]
501 [D loss: 0.999972] [G loss: 1.

674 [D loss: 0.999973] [G loss: 1.000065]
675 [D loss: 0.999971] [G loss: 1.000062]
676 [D loss: 0.999971] [G loss: 1.000067]
677 [D loss: 0.999973] [G loss: 1.000067]
678 [D loss: 0.999971] [G loss: 1.000068]
679 [D loss: 0.999970] [G loss: 1.000066]
680 [D loss: 0.999970] [G loss: 1.000067]
681 [D loss: 0.999971] [G loss: 1.000065]
682 [D loss: 0.999972] [G loss: 1.000066]
683 [D loss: 0.999968] [G loss: 1.000064]
684 [D loss: 0.999970] [G loss: 1.000066]
685 [D loss: 0.999970] [G loss: 1.000067]
686 [D loss: 0.999970] [G loss: 1.000065]
687 [D loss: 0.999970] [G loss: 1.000065]
688 [D loss: 0.999969] [G loss: 1.000065]
689 [D loss: 0.999970] [G loss: 1.000065]
690 [D loss: 0.999971] [G loss: 1.000065]
691 [D loss: 0.999969] [G loss: 1.000067]
692 [D loss: 0.999969] [G loss: 1.000066]
693 [D loss: 0.999970] [G loss: 1.000065]
694 [D loss: 0.999972] [G loss: 1.000066]
695 [D loss: 0.999970] [G loss: 1.000068]
696 [D loss: 0.999970] [G loss: 1.000070]
697 [D loss: 0.999968] [G loss: 1.

...

3165 [D loss: 0.999968] [G loss: 1.000064]
3166 [D loss: 0.999971] [G loss: 1.000062]
3167 [D loss: 0.999972] [G loss: 1.000067]
3168 [D loss: 0.999972] [G loss: 1.000063]
3169 [D loss: 0.999969] [G loss: 1.000064]
3170 [D loss: 0.999970] [G loss: 1.000065]
3171 [D loss: 0.999970] [G loss: 1.000066]
3172 [D loss: 0.999971] [G loss: 1.000067]
3173 [D loss: 0.999969] [G loss: 1.000069]
3174 [D loss: 0.999971] [G loss: 1.000062]
3175 [D loss: 0.999970] [G loss: 1.000066]
3176 [D loss: 0.999971] [G loss: 1.000066]
3177 [D loss: 0.999968] [G loss: 1.000066]
3178 [D loss: 0.999971] [G loss: 1.000065]
3179 [D loss: 0.999970] [G loss: 1.000069]
3180 [D loss: 0.999971] [G loss: 1.000066]
3181 [D loss: 0.999972] [G loss: 1.000063]
3182 [D loss: 0.999970] [G loss: 1.000065]
3183 [D loss: 0.999974] [G loss: 1.000065]
3184 [D loss: 0.999970] [G loss: 1.000066]
3185 [D loss: 0.999970] [G loss: 1.000065]
3186 [D loss: 0.999971] [G loss: 1.000067]
3187 [D loss: 0.999972] [G loss: 1.000063]
3188 [D los

3356 [D loss: 0.999969] [G loss: 1.000067]
3357 [D loss: 0.999970] [G loss: 1.000065]
3358 [D loss: 0.999971] [G loss: 1.000065]
3359 [D loss: 0.999970] [G loss: 1.000067]
3360 [D loss: 0.999970] [G loss: 1.000066]
3361 [D loss: 0.999971] [G loss: 1.000067]
3362 [D loss: 0.999970] [G loss: 1.000067]
3363 [D loss: 0.999970] [G loss: 1.000064]
3364 [D loss: 0.999968] [G loss: 1.000064]
3365 [D loss: 0.999970] [G loss: 1.000065]
3366 [D loss: 0.999970] [G loss: 1.000065]
3367 [D loss: 0.999972] [G loss: 1.000064]
3368 [D loss: 0.999972] [G loss: 1.000065]
3369 [D loss: 0.999969] [G loss: 1.000065]
3370 [D loss: 0.999968] [G loss: 1.000066]
3371 [D loss: 0.999970] [G loss: 1.000068]
3372 [D loss: 0.999971] [G loss: 1.000064]
3373 [D loss: 0.999969] [G loss: 1.000068]
3374 [D loss: 0.999972] [G loss: 1.000064]
3375 [D loss: 0.999972] [G loss: 1.000064]
3376 [D loss: 0.999972] [G loss: 1.000064]
3377 [D loss: 0.999970] [G loss: 1.000065]
3378 [D loss: 0.999972] [G loss: 1.000064]
3379 [D los

3547 [D loss: 0.999972] [G loss: 1.000065]
3548 [D loss: 0.999971] [G loss: 1.000062]
3549 [D loss: 0.999968] [G loss: 1.000067]
3550 [D loss: 0.999970] [G loss: 1.000063]
3551 [D loss: 0.999972] [G loss: 1.000063]
3552 [D loss: 0.999970] [G loss: 1.000064]
3553 [D loss: 0.999970] [G loss: 1.000064]
3554 [D loss: 0.999968] [G loss: 1.000064]
3555 [D loss: 0.999972] [G loss: 1.000067]
3556 [D loss: 0.999972] [G loss: 1.000063]
3557 [D loss: 0.999972] [G loss: 1.000065]
3558 [D loss: 0.999970] [G loss: 1.000061]
3559 [D loss: 0.999968] [G loss: 1.000063]
3560 [D loss: 0.999972] [G loss: 1.000064]
3561 [D loss: 0.999968] [G loss: 1.000067]
3562 [D loss: 0.999971] [G loss: 1.000063]
3563 [D loss: 0.999972] [G loss: 1.000065]
3564 [D loss: 0.999970] [G loss: 1.000066]
3565 [D loss: 0.999972] [G loss: 1.000064]
3566 [D loss: 0.999971] [G loss: 1.000065]
3567 [D loss: 0.999972] [G loss: 1.000066]
3568 [D loss: 0.999970] [G loss: 1.000063]
3569 [D loss: 0.999972] [G loss: 1.000066]
3570 [D los

3738 [D loss: 0.999971] [G loss: 1.000065]
3739 [D loss: 0.999972] [G loss: 1.000067]
3740 [D loss: 0.999969] [G loss: 1.000063]
3741 [D loss: 0.999967] [G loss: 1.000067]
3742 [D loss: 0.999967] [G loss: 1.000063]
3743 [D loss: 0.999971] [G loss: 1.000066]
3744 [D loss: 0.999971] [G loss: 1.000063]
3745 [D loss: 0.999969] [G loss: 1.000065]
3746 [D loss: 0.999972] [G loss: 1.000060]
3747 [D loss: 0.999967] [G loss: 1.000067]
3748 [D loss: 0.999971] [G loss: 1.000066]
3749 [D loss: 0.999971] [G loss: 1.000064]
3750 [D loss: 0.999972] [G loss: 1.000063]
3751 [D loss: 0.999972] [G loss: 1.000063]
3752 [D loss: 0.999972] [G loss: 1.000060]
3753 [D loss: 0.999972] [G loss: 1.000067]
3754 [D loss: 0.999971] [G loss: 1.000063]
3755 [D loss: 0.999969] [G loss: 1.000066]
3756 [D loss: 0.999971] [G loss: 1.000067]
3757 [D loss: 0.999970] [G loss: 1.000067]
3758 [D loss: 0.999972] [G loss: 1.000066]
3759 [D loss: 0.999971] [G loss: 1.000067]
3760 [D loss: 0.999972] [G loss: 1.000064]
3761 [D los

3929 [D loss: 0.999971] [G loss: 1.000066]
3930 [D loss: 0.999971] [G loss: 1.000065]
3931 [D loss: 0.999972] [G loss: 1.000066]
3932 [D loss: 0.999971] [G loss: 1.000066]
3933 [D loss: 0.999971] [G loss: 1.000065]
3934 [D loss: 0.999968] [G loss: 1.000067]
3935 [D loss: 0.999971] [G loss: 1.000066]
3936 [D loss: 0.999969] [G loss: 1.000063]
3937 [D loss: 0.999972] [G loss: 1.000064]
3938 [D loss: 0.999968] [G loss: 1.000069]
3939 [D loss: 0.999972] [G loss: 1.000059]
3940 [D loss: 0.999972] [G loss: 1.000064]
3941 [D loss: 0.999968] [G loss: 1.000066]
3942 [D loss: 0.999969] [G loss: 1.000063]
3943 [D loss: 0.999973] [G loss: 1.000063]
3944 [D loss: 0.999971] [G loss: 1.000066]
3945 [D loss: 0.999969] [G loss: 1.000064]
3946 [D loss: 0.999970] [G loss: 1.000066]
3947 [D loss: 0.999971] [G loss: 1.000069]
3948 [D loss: 0.999970] [G loss: 1.000061]
3949 [D loss: 0.999970] [G loss: 1.000064]
3950 [D loss: 0.999971] [G loss: 1.000065]
3951 [D loss: 0.999970] [G loss: 1.000063]
3952 [D los

In [21]:
noise=np.random.normal(0, 1, (32, 100))
valid=-np.ones((32, 1))


In [22]:
wgan.combined.train_on_batch(noise,valid)

-0.00015029908

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_123 (Conv2D)          (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_72 (LeakyReLU)   (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_72 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_124 (Conv2D)          (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_18 (ZeroPaddi (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_88 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_73 (LeakyReLU)   (None, 8, 8, 64)        

86 [D loss: 0.999962] [G loss: 1.000165]
87 [D loss: 0.999933] [G loss: 1.000184]
88 [D loss: 0.999956] [G loss: 1.000174]
89 [D loss: 0.999975] [G loss: 1.000145]
90 [D loss: 0.999966] [G loss: 1.000138]
91 [D loss: 0.999974] [G loss: 1.000134]
92 [D loss: 0.999965] [G loss: 1.000143]
93 [D loss: 0.999956] [G loss: 1.000136]
94 [D loss: 0.999967] [G loss: 1.000111]
95 [D loss: 0.999958] [G loss: 1.000117]
96 [D loss: 0.999965] [G loss: 1.000109]
97 [D loss: 0.999958] [G loss: 1.000100]
98 [D loss: 0.999970] [G loss: 1.000101]
99 [D loss: 0.999968] [G loss: 1.000092]
100 [D loss: 0.999961] [G loss: 1.000084]
101 [D loss: 0.999965] [G loss: 1.000105]
102 [D loss: 0.999964] [G loss: 1.000106]
103 [D loss: 0.999971] [G loss: 1.000094]
104 [D loss: 0.999967] [G loss: 1.000097]
105 [D loss: 0.999973] [G loss: 1.000098]
106 [D loss: 0.999962] [G loss: 1.000093]
107 [D loss: 0.999979] [G loss: 1.000093]
108 [D loss: 0.999979] [G loss: 1.000091]
109 [D loss: 0.999972] [G loss: 1.000111]
110 [D

282 [D loss: 0.999966] [G loss: 1.000060]
283 [D loss: 0.999973] [G loss: 1.000058]
284 [D loss: 0.999970] [G loss: 1.000058]
285 [D loss: 0.999975] [G loss: 1.000057]
286 [D loss: 0.999956] [G loss: 1.000055]
287 [D loss: 0.999973] [G loss: 1.000068]
288 [D loss: 0.999968] [G loss: 1.000043]
289 [D loss: 0.999971] [G loss: 1.000037]
290 [D loss: 0.999974] [G loss: 1.000058]
291 [D loss: 0.999966] [G loss: 1.000060]
292 [D loss: 0.999965] [G loss: 1.000056]
293 [D loss: 0.999977] [G loss: 1.000058]
294 [D loss: 0.999961] [G loss: 1.000049]
295 [D loss: 0.999968] [G loss: 1.000031]
296 [D loss: 0.999972] [G loss: 1.000068]
297 [D loss: 0.999950] [G loss: 1.000076]
298 [D loss: 0.999971] [G loss: 1.000061]
299 [D loss: 0.999963] [G loss: 1.000055]
300 [D loss: 0.999985] [G loss: 1.000038]
301 [D loss: 0.999964] [G loss: 1.000047]
302 [D loss: 0.999969] [G loss: 1.000045]
303 [D loss: 0.999972] [G loss: 1.000033]
304 [D loss: 0.999961] [G loss: 1.000036]
305 [D loss: 0.999973] [G loss: 1.

478 [D loss: 0.999972] [G loss: 1.000037]
479 [D loss: 0.999981] [G loss: 1.000032]
480 [D loss: 0.999990] [G loss: 1.000017]
481 [D loss: 0.999971] [G loss: 1.000023]
482 [D loss: 0.999970] [G loss: 1.000023]
483 [D loss: 0.999975] [G loss: 1.000054]
484 [D loss: 0.999967] [G loss: 1.000048]
485 [D loss: 0.999964] [G loss: 1.000040]
486 [D loss: 0.999984] [G loss: 1.000048]
487 [D loss: 0.999976] [G loss: 1.000017]
488 [D loss: 0.999964] [G loss: 1.000018]
489 [D loss: 0.999979] [G loss: 1.000047]
490 [D loss: 0.999965] [G loss: 1.000066]
491 [D loss: 0.999970] [G loss: 1.000020]
492 [D loss: 0.999963] [G loss: 1.000053]
493 [D loss: 0.999964] [G loss: 1.000037]
494 [D loss: 0.999981] [G loss: 1.000040]
495 [D loss: 0.999984] [G loss: 1.000018]
496 [D loss: 0.999982] [G loss: 1.000015]
497 [D loss: 0.999977] [G loss: 1.000033]
498 [D loss: 0.999986] [G loss: 1.000021]
499 [D loss: 0.999993] [G loss: 1.000028]
500 [D loss: 0.999970] [G loss: 1.000044]
501 [D loss: 0.999964] [G loss: 1.

674 [D loss: 1.000048] [G loss: 0.999986]
675 [D loss: 1.000021] [G loss: 0.999928]
676 [D loss: 1.000005] [G loss: 0.999983]
677 [D loss: 0.999990] [G loss: 1.000008]
678 [D loss: 1.000007] [G loss: 0.999972]
679 [D loss: 1.000005] [G loss: 0.999968]
680 [D loss: 0.999991] [G loss: 0.999986]
681 [D loss: 0.999994] [G loss: 0.999921]
682 [D loss: 1.000021] [G loss: 0.999944]
683 [D loss: 0.999958] [G loss: 0.999988]
684 [D loss: 0.999989] [G loss: 0.999927]
685 [D loss: 1.000026] [G loss: 0.999948]
686 [D loss: 1.000022] [G loss: 0.999944]
687 [D loss: 1.000030] [G loss: 1.000010]
688 [D loss: 1.000024] [G loss: 0.999997]
689 [D loss: 0.999949] [G loss: 1.000061]
690 [D loss: 0.999977] [G loss: 0.999971]
691 [D loss: 1.000046] [G loss: 0.999947]
692 [D loss: 0.999987] [G loss: 1.000005]
693 [D loss: 1.000031] [G loss: 0.999923]
694 [D loss: 1.000000] [G loss: 0.999934]
695 [D loss: 1.000012] [G loss: 0.999983]
696 [D loss: 1.000010] [G loss: 1.000011]
697 [D loss: 1.000012] [G loss: 0.

...

3165 [D loss: 0.999878] [G loss: 1.000057]
3166 [D loss: 0.999937] [G loss: 1.000051]
3167 [D loss: 0.999963] [G loss: 0.999972]
3168 [D loss: 0.999978] [G loss: 0.999989]
3169 [D loss: 0.999999] [G loss: 0.999987]
3170 [D loss: 0.999923] [G loss: 1.000029]
3171 [D loss: 0.999963] [G loss: 0.999999]
3172 [D loss: 1.000007] [G loss: 0.999973]
3173 [D loss: 0.999937] [G loss: 1.000037]
3174 [D loss: 0.999982] [G loss: 0.999998]
3175 [D loss: 1.000038] [G loss: 0.999959]
3176 [D loss: 0.999970] [G loss: 1.000000]
3177 [D loss: 1.000017] [G loss: 0.999967]
3178 [D loss: 0.999954] [G loss: 0.999955]
3179 [D loss: 0.999947] [G loss: 1.000050]
3180 [D loss: 0.999962] [G loss: 0.999954]
3181 [D loss: 0.999941] [G loss: 0.999993]
3182 [D loss: 0.999944] [G loss: 1.000036]
3183 [D loss: 0.999909] [G loss: 0.999924]
3184 [D loss: 0.999994] [G loss: 0.999921]
3185 [D loss: 1.000010] [G loss: 0.999982]
3186 [D loss: 0.999986] [G loss: 1.000110]
3187 [D loss: 0.999970] [G loss: 1.000105]
3188 [D los

3356 [D loss: 0.999977] [G loss: 0.999925]
3357 [D loss: 0.999956] [G loss: 0.999978]
3358 [D loss: 1.000041] [G loss: 0.999886]
3359 [D loss: 0.999952] [G loss: 1.000040]
3360 [D loss: 0.999926] [G loss: 0.999995]
3361 [D loss: 0.999958] [G loss: 0.999925]
3362 [D loss: 0.999966] [G loss: 0.999978]
3363 [D loss: 0.999934] [G loss: 0.999958]
3364 [D loss: 0.999959] [G loss: 0.999933]
3365 [D loss: 1.000054] [G loss: 0.999915]
3366 [D loss: 1.000024] [G loss: 0.999929]
3367 [D loss: 0.999969] [G loss: 0.999999]
3368 [D loss: 1.000001] [G loss: 0.999958]
3369 [D loss: 0.999978] [G loss: 0.999962]
3370 [D loss: 0.999949] [G loss: 0.999978]
3371 [D loss: 0.999991] [G loss: 0.999850]
3372 [D loss: 0.999937] [G loss: 1.000092]
3373 [D loss: 0.999979] [G loss: 1.000057]
3374 [D loss: 0.999925] [G loss: 1.000080]
3375 [D loss: 1.000019] [G loss: 1.000018]
3376 [D loss: 0.999980] [G loss: 1.000022]
3377 [D loss: 0.999898] [G loss: 1.000025]
3378 [D loss: 0.999949] [G loss: 1.000063]
3379 [D los

3547 [D loss: 1.000000] [G loss: 1.000041]
3548 [D loss: 0.999936] [G loss: 1.000039]
3549 [D loss: 1.000037] [G loss: 0.999983]
3550 [D loss: 0.999968] [G loss: 1.000043]
3551 [D loss: 0.999956] [G loss: 1.000006]
3552 [D loss: 0.999994] [G loss: 1.000007]
3553 [D loss: 0.999967] [G loss: 1.000069]
3554 [D loss: 0.999954] [G loss: 0.999952]
3555 [D loss: 1.000026] [G loss: 0.999974]
3556 [D loss: 0.999953] [G loss: 1.000021]
3557 [D loss: 0.999999] [G loss: 0.999971]
3558 [D loss: 0.999908] [G loss: 1.000012]
3559 [D loss: 0.999960] [G loss: 1.000020]
3560 [D loss: 0.999997] [G loss: 0.999964]
3561 [D loss: 0.999959] [G loss: 0.999997]
3562 [D loss: 0.999988] [G loss: 0.999978]
3563 [D loss: 1.000007] [G loss: 0.999873]
3564 [D loss: 0.999968] [G loss: 0.999959]
3565 [D loss: 1.000004] [G loss: 1.000000]
3566 [D loss: 0.999983] [G loss: 1.000005]
3567 [D loss: 0.999992] [G loss: 0.999985]
3568 [D loss: 0.999949] [G loss: 0.999980]
3569 [D loss: 0.999957] [G loss: 1.000022]
3570 [D los

3738 [D loss: 0.999933] [G loss: 0.999919]
3739 [D loss: 0.999943] [G loss: 1.000022]
3740 [D loss: 1.000048] [G loss: 1.000051]
3741 [D loss: 0.999906] [G loss: 1.000056]
3742 [D loss: 0.999911] [G loss: 1.000029]
3743 [D loss: 1.000033] [G loss: 0.999992]
3744 [D loss: 1.000015] [G loss: 0.999998]
3745 [D loss: 1.000021] [G loss: 0.999937]
3746 [D loss: 0.999980] [G loss: 0.999977]
3747 [D loss: 1.000061] [G loss: 0.999920]
3748 [D loss: 0.999900] [G loss: 1.000045]
3749 [D loss: 0.999964] [G loss: 1.000013]
3750 [D loss: 1.000005] [G loss: 1.000048]
3751 [D loss: 1.000005] [G loss: 0.999924]
3752 [D loss: 1.000008] [G loss: 1.000050]
3753 [D loss: 1.000054] [G loss: 0.999892]
3754 [D loss: 0.999998] [G loss: 0.999910]
3755 [D loss: 0.999947] [G loss: 0.999949]
3756 [D loss: 1.000011] [G loss: 0.999830]
3757 [D loss: 0.999979] [G loss: 0.999864]
3758 [D loss: 0.999861] [G loss: 1.000025]
3759 [D loss: 0.999982] [G loss: 0.999993]
3760 [D loss: 0.999994] [G loss: 0.999909]
3761 [D los

3929 [D loss: 0.999971] [G loss: 0.999991]
3930 [D loss: 0.999940] [G loss: 1.000053]
3931 [D loss: 0.999980] [G loss: 0.999978]
3932 [D loss: 0.999994] [G loss: 0.999977]
3933 [D loss: 0.999969] [G loss: 0.999929]
3934 [D loss: 0.999944] [G loss: 1.000047]
3935 [D loss: 0.999980] [G loss: 1.000028]
3936 [D loss: 0.999962] [G loss: 1.000008]
3937 [D loss: 0.999952] [G loss: 0.999947]
3938 [D loss: 0.999968] [G loss: 1.000017]
3939 [D loss: 1.000012] [G loss: 0.999966]
3940 [D loss: 0.999968] [G loss: 0.999986]
3941 [D loss: 0.999942] [G loss: 1.000012]
3942 [D loss: 0.999945] [G loss: 0.999945]
3943 [D loss: 0.999968] [G loss: 0.999988]
3944 [D loss: 0.999944] [G loss: 1.000019]
3945 [D loss: 0.999951] [G loss: 1.000012]
3946 [D loss: 0.999964] [G loss: 1.000048]
3947 [D loss: 1.000019] [G loss: 0.999991]
3948 [D loss: 0.999995] [G loss: 0.999977]
3949 [D loss: 0.999876] [G loss: 0.999987]
3950 [D loss: 1.000001] [G loss: 0.999965]
3951 [D loss: 1.000024] [G loss: 0.999919]
3952 [D los

In [ ]:
#clipping 0.005 dataset2
#clipping 0.02 dataset3

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_130 (Conv2D)          (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_76 (LeakyReLU)   (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_76 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_131 (Conv2D)          (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_19 (ZeroPaddi (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_93 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_77 (LeakyReLU)   (None, 8, 8, 64)        

86 [D loss: 0.999988] [G loss: 1.033873]
87 [D loss: 1.000763] [G loss: 1.033461]
88 [D loss: 1.002119] [G loss: 1.032621]
89 [D loss: 1.000676] [G loss: 1.032386]
90 [D loss: 1.000908] [G loss: 1.034000]
91 [D loss: 1.001321] [G loss: 1.031486]
92 [D loss: 1.000133] [G loss: 1.031248]
93 [D loss: 1.001026] [G loss: 1.032875]
94 [D loss: 1.000625] [G loss: 1.032281]
95 [D loss: 1.000267] [G loss: 1.031750]
96 [D loss: 1.000989] [G loss: 1.030526]
97 [D loss: 1.000540] [G loss: 1.029212]
98 [D loss: 1.001107] [G loss: 1.031436]
99 [D loss: 0.999814] [G loss: 1.031231]
100 [D loss: 1.001328] [G loss: 1.031086]
101 [D loss: 1.000900] [G loss: 1.030717]
102 [D loss: 1.001066] [G loss: 1.030372]
103 [D loss: 0.999741] [G loss: 1.029724]
104 [D loss: 1.001413] [G loss: 1.029781]
105 [D loss: 1.000486] [G loss: 1.029769]
106 [D loss: 1.001338] [G loss: 1.029174]
107 [D loss: 1.000862] [G loss: 1.029824]
108 [D loss: 1.002211] [G loss: 1.028785]
109 [D loss: 1.001061] [G loss: 1.029047]
110 [D

282 [D loss: 1.000594] [G loss: 1.025922]
283 [D loss: 0.999835] [G loss: 1.027860]
284 [D loss: 1.001866] [G loss: 1.027099]
285 [D loss: 1.002433] [G loss: 1.027652]
286 [D loss: 1.000759] [G loss: 1.026922]
287 [D loss: 0.998733] [G loss: 1.025899]
288 [D loss: 0.999994] [G loss: 1.026280]
289 [D loss: 1.000837] [G loss: 1.023921]
290 [D loss: 1.000880] [G loss: 1.024126]
291 [D loss: 1.002564] [G loss: 1.023866]
292 [D loss: 1.000437] [G loss: 1.022888]
293 [D loss: 0.999341] [G loss: 1.022718]
294 [D loss: 0.999982] [G loss: 1.022769]
295 [D loss: 1.001189] [G loss: 1.022553]
296 [D loss: 0.999835] [G loss: 1.025177]
297 [D loss: 0.999134] [G loss: 1.024351]
298 [D loss: 1.001104] [G loss: 1.021520]
299 [D loss: 1.001325] [G loss: 1.022051]
300 [D loss: 1.001589] [G loss: 1.021245]
301 [D loss: 1.001154] [G loss: 1.021471]
302 [D loss: 1.000452] [G loss: 1.024048]
303 [D loss: 1.000979] [G loss: 1.022610]
304 [D loss: 1.000582] [G loss: 1.021753]
305 [D loss: 1.001735] [G loss: 1.

478 [D loss: 1.000571] [G loss: 1.006506]
479 [D loss: 1.001629] [G loss: 1.005088]
480 [D loss: 1.000479] [G loss: 1.003350]
481 [D loss: 1.004001] [G loss: 1.004987]
482 [D loss: 1.001817] [G loss: 1.004623]
483 [D loss: 1.001884] [G loss: 1.005954]
484 [D loss: 1.001599] [G loss: 1.005977]
485 [D loss: 1.001178] [G loss: 1.007533]
486 [D loss: 1.002003] [G loss: 1.005505]
487 [D loss: 1.002686] [G loss: 1.007459]
488 [D loss: 1.000727] [G loss: 1.007466]
489 [D loss: 1.000467] [G loss: 1.004441]
490 [D loss: 1.002463] [G loss: 1.008149]
491 [D loss: 1.000099] [G loss: 1.008744]
492 [D loss: 1.001268] [G loss: 1.008366]
493 [D loss: 0.999519] [G loss: 1.004643]
494 [D loss: 1.001881] [G loss: 1.006498]
495 [D loss: 1.001592] [G loss: 1.006939]
496 [D loss: 1.000909] [G loss: 1.004155]
497 [D loss: 1.001074] [G loss: 1.004356]
498 [D loss: 1.002036] [G loss: 1.003369]
499 [D loss: 1.001461] [G loss: 1.005293]
500 [D loss: 1.002020] [G loss: 1.005387]
501 [D loss: 1.002238] [G loss: 1.

674 [D loss: 1.009466] [G loss: 0.990894]
675 [D loss: 1.006267] [G loss: 0.988162]
676 [D loss: 1.007480] [G loss: 0.989681]
677 [D loss: 1.002610] [G loss: 0.997033]
678 [D loss: 1.000701] [G loss: 0.992255]
679 [D loss: 1.003106] [G loss: 0.985880]
680 [D loss: 1.011298] [G loss: 0.987304]
681 [D loss: 1.004126] [G loss: 0.989335]
682 [D loss: 1.009130] [G loss: 0.988190]
683 [D loss: 1.006961] [G loss: 0.989270]
684 [D loss: 1.005904] [G loss: 0.995703]
685 [D loss: 1.008139] [G loss: 0.989569]
686 [D loss: 1.003975] [G loss: 0.988540]
687 [D loss: 1.001222] [G loss: 0.990393]
688 [D loss: 1.010084] [G loss: 0.987182]
689 [D loss: 1.005469] [G loss: 0.991716]
690 [D loss: 1.003927] [G loss: 0.985965]
691 [D loss: 1.007889] [G loss: 0.985619]
692 [D loss: 1.003516] [G loss: 0.989829]
693 [D loss: 1.002467] [G loss: 0.987327]
694 [D loss: 1.003952] [G loss: 0.984183]
695 [D loss: 1.002780] [G loss: 0.989137]
696 [D loss: 1.005050] [G loss: 0.987955]
697 [D loss: 1.006513] [G loss: 0.

...

3165 [D loss: 1.003366] [G loss: 0.993942]
3166 [D loss: 1.006638] [G loss: 0.982634]
3167 [D loss: 0.997311] [G loss: 0.991978]
3168 [D loss: 1.010316] [G loss: 0.980192]
3169 [D loss: 1.007343] [G loss: 0.991072]
3170 [D loss: 1.009091] [G loss: 0.998277]
3171 [D loss: 1.001334] [G loss: 0.974246]
3172 [D loss: 1.009840] [G loss: 0.987052]
3173 [D loss: 1.003747] [G loss: 1.000732]
3174 [D loss: 1.002485] [G loss: 0.987112]
3175 [D loss: 1.001950] [G loss: 0.979258]
3176 [D loss: 1.012929] [G loss: 0.995143]
3177 [D loss: 1.009669] [G loss: 0.993217]
3178 [D loss: 1.016298] [G loss: 0.972605]
3179 [D loss: 0.999219] [G loss: 0.989746]
3180 [D loss: 1.005432] [G loss: 0.986651]
3181 [D loss: 1.014511] [G loss: 0.989437]
3182 [D loss: 1.009359] [G loss: 0.998748]
3183 [D loss: 0.989467] [G loss: 0.976246]
3184 [D loss: 1.000707] [G loss: 1.001982]
3185 [D loss: 0.995786] [G loss: 0.995957]
3186 [D loss: 1.006958] [G loss: 1.001814]
3187 [D loss: 1.000301] [G loss: 0.988014]
3188 [D los

3356 [D loss: 0.999185] [G loss: 0.990166]
3357 [D loss: 1.002681] [G loss: 1.004066]
3358 [D loss: 1.010885] [G loss: 0.987540]
3359 [D loss: 1.002610] [G loss: 0.989526]
3360 [D loss: 1.006746] [G loss: 0.991010]
3361 [D loss: 1.005797] [G loss: 1.001209]
3362 [D loss: 1.007796] [G loss: 0.983150]
3363 [D loss: 1.002673] [G loss: 0.982914]
3364 [D loss: 1.021732] [G loss: 0.991075]
3365 [D loss: 1.000883] [G loss: 0.996927]
3366 [D loss: 1.008182] [G loss: 0.984092]
3367 [D loss: 0.982598] [G loss: 0.990508]
3368 [D loss: 1.004679] [G loss: 1.002138]
3369 [D loss: 0.998027] [G loss: 0.988132]
3370 [D loss: 0.997904] [G loss: 0.990872]
3371 [D loss: 1.023639] [G loss: 0.965475]
3372 [D loss: 1.000874] [G loss: 0.987161]
3373 [D loss: 1.001252] [G loss: 0.994640]
3374 [D loss: 1.010879] [G loss: 0.999714]
3375 [D loss: 1.005584] [G loss: 0.998195]
3376 [D loss: 1.001983] [G loss: 0.991780]
3377 [D loss: 1.012174] [G loss: 0.972210]
3378 [D loss: 0.995882] [G loss: 0.998148]
3379 [D los

3547 [D loss: 1.002312] [G loss: 0.995781]
3548 [D loss: 1.003864] [G loss: 0.984019]
3549 [D loss: 1.017065] [G loss: 0.991751]
3550 [D loss: 1.006613] [G loss: 0.976344]
3551 [D loss: 1.000587] [G loss: 0.990770]
3552 [D loss: 1.011870] [G loss: 0.990975]
3553 [D loss: 1.007017] [G loss: 0.994779]
3554 [D loss: 0.996035] [G loss: 0.980771]
3555 [D loss: 1.013081] [G loss: 0.986236]
3556 [D loss: 1.012264] [G loss: 0.968182]
3557 [D loss: 1.000302] [G loss: 0.998013]
3558 [D loss: 1.006342] [G loss: 0.977299]
3559 [D loss: 1.013086] [G loss: 0.997602]
3560 [D loss: 1.003014] [G loss: 0.995081]
3561 [D loss: 0.994304] [G loss: 0.991298]
3562 [D loss: 1.001919] [G loss: 0.995277]
3563 [D loss: 1.001392] [G loss: 1.000338]
3564 [D loss: 1.005503] [G loss: 0.986447]
3565 [D loss: 1.012928] [G loss: 0.977523]
3566 [D loss: 0.993091] [G loss: 1.000470]
3567 [D loss: 1.006682] [G loss: 0.967712]
3568 [D loss: 1.012220] [G loss: 0.990506]
3569 [D loss: 1.010213] [G loss: 0.995729]
3570 [D los

3738 [D loss: 1.008362] [G loss: 0.986457]
3739 [D loss: 0.998690] [G loss: 0.994555]
3740 [D loss: 0.994306] [G loss: 0.993863]
3741 [D loss: 1.013360] [G loss: 1.011479]
3742 [D loss: 1.005444] [G loss: 0.998178]
3743 [D loss: 1.008448] [G loss: 0.969545]
3744 [D loss: 0.998504] [G loss: 0.989209]
3745 [D loss: 1.007743] [G loss: 0.981592]
3746 [D loss: 0.994943] [G loss: 1.003189]
3747 [D loss: 0.991008] [G loss: 1.017210]
3748 [D loss: 0.999226] [G loss: 0.994292]
3749 [D loss: 1.009278] [G loss: 0.985037]
3750 [D loss: 1.010111] [G loss: 0.982242]
3751 [D loss: 0.999522] [G loss: 0.990928]
3752 [D loss: 0.992948] [G loss: 0.992796]
3753 [D loss: 1.006547] [G loss: 1.005248]
3754 [D loss: 0.991203] [G loss: 0.986283]
3755 [D loss: 0.996247] [G loss: 0.998580]
3756 [D loss: 0.996098] [G loss: 0.982422]
3757 [D loss: 1.005462] [G loss: 1.013242]
3758 [D loss: 0.999616] [G loss: 0.976604]
3759 [D loss: 1.004854] [G loss: 1.001478]
3760 [D loss: 1.009683] [G loss: 1.002642]
3761 [D los

3929 [D loss: 0.992533] [G loss: 0.994016]
3930 [D loss: 1.006131] [G loss: 0.999877]
3931 [D loss: 1.002193] [G loss: 0.995838]
3932 [D loss: 1.010473] [G loss: 0.993145]
3933 [D loss: 0.998043] [G loss: 0.994425]
3934 [D loss: 1.012852] [G loss: 0.991471]
3935 [D loss: 0.998516] [G loss: 0.992780]
3936 [D loss: 0.997369] [G loss: 0.998520]
3937 [D loss: 1.011492] [G loss: 0.988490]
3938 [D loss: 1.007536] [G loss: 0.985897]
3939 [D loss: 1.021298] [G loss: 0.995089]
3940 [D loss: 1.003610] [G loss: 1.003313]
3941 [D loss: 0.994540] [G loss: 0.990026]
3942 [D loss: 1.000548] [G loss: 0.981027]
3943 [D loss: 1.002067] [G loss: 0.997380]
3944 [D loss: 1.018454] [G loss: 0.990466]
3945 [D loss: 0.990400] [G loss: 0.992322]
3946 [D loss: 1.003822] [G loss: 0.972166]
3947 [D loss: 1.012974] [G loss: 0.981604]
3948 [D loss: 1.010978] [G loss: 0.993247]
3949 [D loss: 0.999720] [G loss: 0.981008]
3950 [D loss: 1.003758] [G loss: 1.004651]
3951 [D loss: 1.000872] [G loss: 0.993378]
3952 [D los